原文： http://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html

Translator: 陳胤泰 YinTaiChen


Autograd: 自動微分
===================================

在 PyTorch 中，所有神經網路都以 "autograd" 這個套組 （package）為核心。
讓我們先簡要地了解一下這一點，再來訓練我們的第一個神經網路。


"autograd" 套組提供對所有張量運算的自動微分。它是個藉由運行來定義（define-by-run）的框架，這代表反向傳播是由你的程式碼如何運行來定義的，這也代表每一個迴圈可以各自不同。

讓我們以比較簡單的用語、透過一些例子來看看。

## Variable
--------
``autograd.Variable``是這個套組的核心類別。它將一個張量包裹起來，並支援幾乎所有定義在張量上的運算。一旦你完成了所有運算，你可以呼叫 ``.backward()`` 使所有梯度自動被計算好。

你可以透過``.data``屬性來取用 Variable 內的張量。至於該 Variable 相關聯的梯度，會被累積存放在``.grad``這個屬性之中。

![title](http://pytorch.org/tutorials/_images/Variable.png)

還有一個對 autograd 的實作而言十分重要的類別：``Function``。


``Variable``與``Function``彼此緊密關聯，共同組成了一個無環圖（acyclic graph），而該圖編碼了完整的運算歷史。
每一個 variable 都有一個``.grad_fn``屬性參考了創造該 ``Variable`` 的 ``Function``。（使用者自行創造的 Variable 是例外——它的``grad_fn``的值是``None``）。


如果你想計算導數 （derivative），你可對一個``Variable``呼叫``.backward()``。
如果``Variable``是個純量（亦即，它的資料只有一個數值），你不須明定任何傳遞給它的引數。
然而，如果它具有更多數值，你需要明定``grad_output``這個引數，藉此傳遞一個大小相符的張量。

In [1]:
import torch
from torch.autograd import Variable

創建一個 variable:



In [2]:
x = Variable(torch.ones(2, 2), requires_grad=True)
print(x)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]



執行一個 variable 的運算:



In [3]:
y = x + 2
print(y)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



``y`` 由運算後的結果賦值而得，所以它的``grad_fn``有值。



In [4]:
print(y.grad_fn)

對 y 做更多的運算。



In [5]:
z = y * y * 3
out = z.mean()

print(z, out)

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
 Variable containing:
 27
[torch.FloatTensor of size 1]



梯度(Gradient)
---------
讓我們進行反向傳播。
``out.backward()`` 與``out.backward(torch.Tensor([1.0]))``在此是等價的。



In [6]:
out.backward()

打印梯度 d(out)/dx




In [7]:
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



你應該會得到一個值為``4.5``的矩陣。令``out``*Variable* 為“$o$”。
我們便得到 $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ 以及 $z_i\bigr\rvert_{x_i=1} = 27$。
如此一來，
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$，且由此可知
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



你可以用 autograd 做點很瘋狂的事!



In [8]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

Variable containing:
-913.6132
-411.7198
 307.4206
[torch.FloatTensor of size 3]



In [9]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

Variable containing:
  51.2000
 512.0000
   0.0512
[torch.FloatTensor of size 3]



**稍後閱讀:**

``Variable``與``Function``的文件：
http://pytorch.org/docs/autograd

